# RegionAI Exploration Notebook

This notebook demonstrates the core concepts of RegionAI - representing concepts as regions rather than points in embedding space.

In [ ]:
# Setup
import sys
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
from regionai.geometry.box2d import Box2D
from regionai.spaces.concept_space_2d import ConceptSpace2D
from regionai.visualization.plotter import ConceptPlotter
from regionai.visualization.interactive_plotter import InteractivePlotter, VisualizationMode
from regionai.visualization.selection import SelectionManager, SelectionMode, HighlightStrategy
from regionai.analysis.relationships import RelationshipAnalyzer

%matplotlib widget

## 1. Basic Concepts: Regions vs Points

Traditional embeddings represent concepts as single points. RegionAI represents them as bounded regions.

In [ ]:
# Create a simple concept space
space = ConceptSpace2D()

# Add concepts with different levels of specificity
space.add_region("ENTITY", Box2D(0, 0, 100, 100))  # Most general
space.add_region("PHYSICAL", Box2D(5, 5, 95, 95))  # Slightly more specific
space.add_region("LIVING", Box2D(10, 10, 60, 60))  # Living things
space.add_region("ANIMAL", Box2D(15, 15, 55, 55))  # Animals
space.add_region("DOG", Box2D(20, 20, 35, 35))    # Specific animal
space.add_region("CAT", Box2D(40, 40, 50, 50))    # Another specific animal

# Visualize
plotter = ConceptPlotter()
fig = plotter.plot(space, title="Concept Hierarchy", show_hierarchy=True)
plt.show()

## 2. Hierarchical Relationships

Containment represents "is-a" relationships naturally.

In [ ]:
# Analyze the hierarchical relationships
analyzer = RelationshipAnalyzer(space)
chains = analyzer.find_hierarchical_chains()

print("Hierarchical chains found:")
for chain in chains:
    print(f"  {chain}")

# Check specific relationships
dog_region = space.get_region("DOG")
animal_region = space.get_region("ANIMAL")

print(f"\nDOG is contained in ANIMAL: {animal_region.contains(dog_region)}")
print(f"DOG volume: {dog_region.volume():.1f}")
print(f"ANIMAL volume: {animal_region.volume():.1f}")

## 3. Multiple Inheritance Through Overlaps

Overlapping regions naturally represent concepts that belong to multiple categories.

In [ ]:
# Create a space with overlapping concepts
overlap_space = ConceptSpace2D()

# Base categories
overlap_space.add_region("ANIMAL", Box2D(10, 10, 70, 70))
overlap_space.add_region("FLYING_THING", Box2D(40, 10, 100, 70))
overlap_space.add_region("AQUATIC", Box2D(10, 40, 70, 100))

# Things with multiple inheritance
overlap_space.add_region("BIRD", Box2D(45, 15, 65, 35))  # Animal + Flying
overlap_space.add_region("BAT", Box2D(50, 45, 60, 55))   # Animal + Flying
overlap_space.add_region("FISH", Box2D(15, 45, 35, 65))  # Animal + Aquatic
overlap_space.add_region("PENGUIN", Box2D(35, 50, 45, 60))  # Animal + Aquatic (flightless bird)

# Visualize
fig = plotter.plot(overlap_space, title="Multiple Inheritance via Overlaps")
plt.show()

# Analyze overlaps
for concept in ["BIRD", "BAT", "FISH", "PENGUIN"]:
    region = overlap_space.get_region(concept)
    containers = overlap_space.find_containing_regions(region)
    print(f"{concept} belongs to: {', '.join(containers)}")

## 4. Interactive Exploration

Use the interactive plotter to explore relationships dynamically.

In [ ]:
# Create an interactive plot
interactive_plotter = InteractivePlotter(overlap_space)
fig = interactive_plotter.create_interactive_plot(title="Interactive Concept Explorer")

print("Keyboard shortcuts:")
print("  Click: Select/deselect region")
print("  ESC: Clear selection")
print("  H: Hierarchy mode")
print("  O: Overlap mode")
print("  D: Debug mode")
print("  TAB: Cycle through regions")

plt.show()

## 5. Concept Space Analysis

Analyze the structure and relationships in the concept space.

In [ ]:
# Perform comprehensive analysis
analyzer = RelationshipAnalyzer(overlap_space)
results = analyzer.analyze_all()

print("=== Concept Space Statistics ===")
for key, value in results.statistics.items():
    print(f"{key}: {value:.2f}")

print("\n=== Hierarchical Chains ===")
for chain in results.hierarchies:
    print(f"  {chain}")

# Calculate conceptual distances
print("\n=== Conceptual Distances ===")
concepts = ["BIRD", "BAT", "FISH", "PENGUIN"]
for i, c1 in enumerate(concepts):
    for c2 in concepts[i+1:]:
        distance = analyzer.find_conceptual_distance(c1, c2)
        print(f"Distance {c1} <-> {c2}: {distance:.2f}")

## 6. Creating Complex Hierarchies

Build more sophisticated concept spaces representing real-world knowledge.

In [ ]:
# Create a knowledge hierarchy
knowledge_space = ConceptSpace2D()

# Top level: Everything
knowledge_space.add_region("KNOWLEDGE", Box2D(0, 0, 200, 200))

# Major branches
knowledge_space.add_region("SCIENCE", Box2D(10, 10, 90, 190))
knowledge_space.add_region("ARTS", Box2D(110, 10, 190, 190))

# Science subcategories
knowledge_space.add_region("PHYSICS", Box2D(15, 15, 45, 90))
knowledge_space.add_region("CHEMISTRY", Box2D(50, 15, 85, 90))
knowledge_space.add_region("BIOLOGY", Box2D(15, 100, 85, 180))

# Interdisciplinary (overlapping)
knowledge_space.add_region("BIOCHEMISTRY", Box2D(45, 95, 75, 125))  # Chemistry + Biology
knowledge_space.add_region("BIOPHYSICS", Box2D(20, 85, 50, 115))    # Physics + Biology

# Arts subcategories  
knowledge_space.add_region("VISUAL_ARTS", Box2D(115, 15, 185, 90))
knowledge_space.add_region("MUSIC", Box2D(115, 100, 185, 180))

# Cross-domain (Science + Arts)
knowledge_space.add_region("VISUALIZATION", Box2D(85, 50, 115, 80))  # Science + Visual Arts
knowledge_space.add_region("SONIFICATION", Box2D(85, 120, 115, 150))  # Science + Music

# Create interactive visualization
knowledge_plotter = InteractivePlotter(knowledge_space)
fig = knowledge_plotter.create_interactive_plot(title="Knowledge Domain Hierarchy")
plt.show()

## 7. Programmatic Interaction

Use the SelectionManager for programmatic control over selections.

In [ ]:
# Create selection manager
selection_mgr = SelectionManager(knowledge_space, mode=SelectionMode.MULTIPLE)
selection_mgr.set_highlight_strategy(HighlightStrategy.RELATED)

# Select interdisciplinary fields
print("Selecting interdisciplinary fields...")
selection_mgr.select_region("BIOCHEMISTRY")
selection_mgr.select_region("VISUALIZATION")

# Get selection info
info = selection_mgr.get_selection_info()
print("\nSelected regions:", info['selected'])
print("Highlighted regions:", info['highlighted'])

# Explore relationships
print("\nRelationships for selected regions:")
for region_name, rels in info['relationships'].items():
    print(f"\n{region_name}:")
    for rel_type, related in rels.items():
        if related:
            print(f"  {rel_type}: {', '.join(related)}")

## 8. Future Directions

This 2D visualization is just the beginning. The same concepts extend to:

1. **Higher Dimensions**: Real embeddings would use hundreds of dimensions
2. **Dynamic Regions**: Regions that change based on context
3. **Probabilistic Regions**: Fuzzy boundaries for uncertain concepts
4. **Learning**: Discovering regions from data rather than manual definition

The key insight is that representing concepts as regions rather than points enables:
- Explicit uncertainty representation
- Natural hierarchies through containment
- Multiple inheritance through overlaps
- Graded membership based on position within regions